In [1]:
import googleapiclient.discovery
import pandas as pd
import os

import requests_cache
requests_cache.install_cache("yt_api")


In [2]:
def connect_yt_data_api():
    api_service_name = "youtube"
    api_version = "v3"

    with open("../../google_api_key.key") as file:
        DEVELOPER_KEY = file.read()

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)
    
    return youtube

def get_comments(videoId:str=None, allThreadsRelatedToChannelId:str=None, pageToken=None):
    if not videoId and not allThreadsRelatedToChannelId:
        raise Exception("no video id or channelId specified.")

    
    youtube = connect_yt_data_api()

    request = youtube.commentThreads().list(
        part="snippet",
        allThreadsRelatedToChannelId=allThreadsRelatedToChannelId,
        videoId=videoId,
        maxResults=100,
        pageToken=pageToken
    )

    response = request.execute()
    return response


In [4]:
def get_channel_id(channelName):

    youtube = connect_yt_data_api()
    
    request = youtube.channels().list(
        part="statistics",
        forUsername="TeslaMotors"
    )
    channels = request.execute()

    return channels["items"][0]["id"]

In [5]:
get_channel_id("TeslaMotors")

'UC5WjFrtBdufl6CZojX3D8dQ'

In [10]:
def extract_comments(videoId, pages=1):
    parsed_comments = []
    
    pageToken=None
    
    for _ in range(pages):
        comments = get_comments(videoId=videoId, pageToken=pageToken)
        pageToken = comments["nextPageToken"]
        
        for comment in comments["items"]:
            tplc = comment["snippet"]["topLevelComment"]["snippet"]

            parsed_comment = {
                "textOriginal":tplc["textOriginal"],
                "authorDisplayName":tplc["authorDisplayName"],
                "publishedAt":tplc["publishedAt"],
                "likeCount":tplc["likeCount"]
            }
            
            parsed_comments.append(parsed_comment)

        
    return parsed_comments

In [11]:
cs = extract_comments("eelCWETObJ8", pages=2)

In [14]:
cs

[{'textOriginal': 'Alo bre vracaj mikiju kanal',
  'authorDisplayName': 'Ogi Mi',
  'publishedAt': '2023-02-16T13:26:48Z',
  'likeCount': 1},
 {'textOriginal': 'Love it love it love it. But where are the ladies?',
  'authorDisplayName': 'John Burger',
  'publishedAt': '2023-02-16T13:03:30Z',
  'likeCount': 0},
 {'textOriginal': '😂😂😂.... Tesla Insurance in MA please!! 🙏 🙏 🙏',
  'authorDisplayName': 'John Grimble',
  'publishedAt': '2023-02-16T12:52:08Z',
  'likeCount': 0},
 {'textOriginal': 'MIHAJLOOO',
  'authorDisplayName': 'Uros Peric',
  'publishedAt': '2023-02-16T12:18:18Z',
  'likeCount': 1},
 {'textOriginal': 'Mater ti poljubi',
  'authorDisplayName': 'fran rili zagorec',
  'publishedAt': '2023-02-16T12:13:48Z',
  'likeCount': 0},
 {'textOriginal': 'HACKER GIVE BACK THE CHANNEL',
  'authorDisplayName': 'vesonfire',
  'publishedAt': '2023-02-16T11:23:35Z',
  'likeCount': 1},
 {'textOriginal': 'Hakovali ste mihajlov kanal sram vas bilo',
  'authorDisplayName': 'DraganXD',
  'publis